## Preparation

In [1]:
import geopandas as gpd
import pandas as pd
import os
import glob
import datetime
import numpy as np
import rasterio
import rasterio.mask
from funcs import GIS_function as gis

In [2]:
# read inputs from spreadsheet
working_dr = os.getcwd()
data_dr = os.path.join(working_dr, 'data')
inputs_dr = os.path.join(working_dr, 'inputs.csv')
inputs_df = pd.read_csv(inputs_dr)

In [3]:
# choose a season
# by index
index = 0
row = inputs_df.iloc[index]
# # by name
# name = '2020_2021_wheat'
# row = inputs_df.loc[inputs_df['season'] == name].iloc[0]
season = row['season']
project_area_dr = row['project_area']
field_area_dr = row['field_area']
start_date = row['start_date']
end_date = row['end_date']
project_gdf = gpd.read_file(project_area_dr) #read shapefile
field_gdf = gpd.read_file(field_area_dr) #read shapefile

In [4]:
shp_name = os.path.basename(project_area_dr).split('.')[0]
project_dr = os.path.join(working_dr, 'data', shp_name)
season_dr = os.path.join(project_dr, f'season_{season}')
dekads_dr = os.path.join(season_dr, "dekads")
WaPOR_dr = os.path.join(season_dr, 'WaPOR_data')
et_folder = os.path.join(season_dr, 'ET')
ndvi_folder = os.path.join(season_dr, 'ndvi')
rgb_folder = os.path.join(season_dr, 'rgb')

## Resample WaPOR data

In [7]:
dekads = sorted(os.listdir(dekads_dr))
template = glob.glob(os.path.join(season_dr, dekads[0], "ETa", "tifs")+"/*.tif")[0]
ret_tifs = glob.glob(os.path.join(WaPOR_dr, 'L1-RET-E')+"/*.tif")
resample = gis.MatchProjResNDV(
          template, ret_tifs, os.path.join(WaPOR_dr, 'L1-RET-E_resampled'),
          resample = 'near', dtype = 'float32')

## fill missing data with WaPOR data

In [5]:
dekads = sorted(os.listdir(dekads_dr))
first_dekad = datetime.datetime.strptime(dekads[0], "%Y-%m-%d")
last_dekad = datetime.datetime.strptime(dekads[-1], "%Y-%m-%d")
first_dekad, last_dekad
desired_days = [1, 11, 21]
date_list = []
current_date = first_dekad
while current_date <= last_dekad:
  if current_date.day in desired_days:
    date_list.append(current_date)
  current_date += datetime.timedelta(days=1)
date_list = [i.strftime("%Y-%m-%d") for i in date_list]
lost_dekads = list(set(date_list).difference(set(dekads)))
template = glob.glob(os.path.join(dekads_dr, '2024-07-01', "ETa", "tifs")+"/*.tif")[0]
if len(lost_dekads) != 0:
  wapor_et = sorted(os.listdir(os.path.join(WaPOR_dr, "L2-AETI-D")))
  for dekad in lost_dekads:
    for et_tif in wapor_et:
      if dekad in et_tif:
        resample = gis.MatchProjResNDV(
          template, [os.path.join(WaPOR_dr, "L2-AETI-D", et_tif)], os.path.join(dekads_dr, dekad, "ETa"),
          resample = 'near', dtype = 'float32')
        os.rename(resample[0],
          resample[0].replace(
            os.path.basename(resample[0]),
            dekad + ".tif"
            ))

## Biomass from NPP WaPOR data

In [5]:
# Update the crop parameters specic to the crop
MC = 0.14  # moisture content, dry matter over freshbiomass
fc = 1  # Light use efficiency correction factor
AOT= 0.7  # above ground over total biomass production ratio(AOT)

In [16]:
template = glob.glob(os.path.join(WaPOR_dr, "L2-NPP-D")+'/*.tif')[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(template)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

wapor_npp = sorted(glob.glob(os.path.join(WaPOR_dr, "L2-NPP-D")+'/*.tif'))
for npp_tif in wapor_npp:
  dekad = os.path.basename(npp_tif).split('.')[0].split('_')[-1]
  NPP  = gis.OpenAsArray(npp_tif, nan_values=True)
  AGBM = (AOT * fc * (NPP * 22.222 / (1 - MC))) / 1000  # Above ground biomass, 1000 is to covert from kg to ton
  os.makedirs(os.path.join(WaPOR_dr, "L2-biomass-D"), exist_ok=True)
  biomass_dr    = os.path.join(WaPOR_dr, "L2-biomass-D", f"{dekad}.tif")
  gis.CreateGeoTiff(biomass_dr, AGBM, driver, NDV, xsize, ysize, GeoT, Projection)
  
template = glob.glob(os.path.join(dekads_dr, '2024-07-01', "ETa", "tifs")+"/*.tif")[0]
wapor_biomass = sorted(glob.glob(os.path.join(WaPOR_dr, "L2-biomass-D")+'/*.tif'))
for biomass_tif in wapor_biomass:
    dekad = os.path.basename(biomass_tif).split('.')[0].split('_')[-1]
    resample = gis.MatchProjResNDV(
      template, [os.path.join(WaPOR_dr, "L2-biomass-D", biomass_tif)], os.path.join(dekads_dr, dekad, "biomass"),
      resample = 'near', dtype = 'float32')
    os.rename(resample[0],
      resample[0].replace(
        os.path.basename(resample[0]),
        dekad + ".tif"
        ))

## Kc (ETa/ETp) tifs

In [29]:
from funcs.export import kc_tifs

kc_tifs(season_dr)

project_RET_L1-RET-E_NONE_day_2024-07-01.tif
project_RET_L1-RET-E_NONE_day_2024-07-09.tif


## Dataframes

In [6]:
from funcs.write_df import write_dfs

In [7]:
template = glob.glob(os.path.join(dekads_dr, "2024-07-01", "ETa", "tifs")+"/*.tif")[0]
src = rasterio.open(template)
field_gdf = field_gdf.to_crs(src.crs)
project_gdf = project_gdf.to_crs(src.crs)
project_gdf['id'] = 0

write_dfs(season_dr, field_gdf)
write_dfs(season_dr, project_gdf)

## Dekadal images

In [37]:
from funcs.export import dekads_tifs

dekads_tifs(season_dr, project_gdf)